In [1]:
import tensorflow as tf
#from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/luokui/mnist/data/",one_hot=True)

/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /Users/luokui/mnist/data/train-images-idx3-ubyte.gz
Extracting /Users/luokui/mnist/data/train-labels-idx1-ubyte.gz
Extracting /Users/luokui/mnist/data/t10k-images-idx3-ubyte.gz
Extracting /Users/luokui/mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np
import sklearn.preprocessing as prep


In [3]:
def xavier_init(fan_in,fan_out,constant):
    low = -constant * tf.sqrt(6.0/(fan_in+fan_out))
    high = constant ( tf.sqrt(6.0/(fan_in+fan_out)))
    return tf.random_uniform((fan_in,fan_out),minval=low,maxval=high,dtype=tf.float32)

In [7]:
class AdditiveGaussianAutoencoder(object):
    def _init_(self,n_input,n_hidden,transfer_function=tf.nn.softplus,optimizer=tf.train.AdamOptimizer(),scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
    
        self.x = tf.placeholder(tf.float32, [None,self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x+scale*tf.random_normal((n_input,)),self.weights['w1']),
                                                             self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden,self.weights['w2']),tf.weights['b2'])
        self.loss =0.5*tf.reduce_mean(tf.square(tf.subtract(self.x,self.reconstruction)))
        self.optimizer = optimizer.minimize(self.loss)
        
        self.init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(self.init)
        
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.in_put,self.n_hidder))
        all_weights['b1'] = tf.Variable(tf.zeros([self.hidden],dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.hidden,self.n_input],dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],dtype=tf.float32))
        return all_weights
    
    def partial_fit(self,x):
        cost, opt = self.sess.run((self.cost,self.optimizer),feed_dict = {self.x:x,self.scale:self.training_scale})
        return cost
    
    def calc_total_cost(self,x):
        return self.sess.run(self.cost,feed_dict={self.x:x,self.scale:self.training_scale})
    
    def transform(self,x):
        return self.sess.run(self.hidden,feed_dict={self.x:x,self.scale:self.training_scale})
    
    def generate(self,hidden):
        if hidden is None:
            hidden = np.random.normal(size=self.weights['b1'])
        return self.sess.run(self.reconstruction,feed_dict={self.hidden:hidden})
    
    def reconstruct(self,x):
        return self.sess.run(self.reconstruction,feed_dict={self.x:x,self.scale:self.training_scale})
    
    def getweights(self):
        return self.sess.run(self.weights['w1'])
    
    def getbias(self):
        return self.sess.run(self.weights['b1'])
    

In [8]:
def standard_scale(x_train,x_test):
    preprocessor = prep.StandardScaler().fit(x_train)
    x_train = preprocessor.transform(x_train)
    x_test = preprocessor.transform(x_test)
    return x_train,x_test

def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data)-batch_size) #在 0~len(data)-batch_size 之间任意选择一个数字
    return data[start_index:(start_index + batch_size)] #选取之间的数字

x_train,x_test = standard_scale(mnist.train.images,mnist.test.images)
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1


In [9]:
autoencoder = AdditiveGaussianAutoencoder(n_input = 784,n_hidden=200)

TypeError: object() takes no parameters